Список нужных классов

In [8]:
class_list = ['person', 'car', 'motorcycle', 'bus', 'truck']

Список классов датасета

In [9]:
coco_classes = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']

Список классов не подлежащих обучению

In [10]:
def list_exclude(a, b):
    return [element for element in a if element not in b]

exclude_list = list_exclude(coco_classes, class_list)
exclude_list[:10]

['bicycle',
 'airplane',
 'train',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird']

# Список индексов подлежащих обучению

In [11]:
def list_index_to_exclude(a, b):
    exclude = []
    for each in b:
        print(each)
        for index, element in enumerate(a):
            # print(index, element, sep=': ')
            if each == element:
                exclude.append(str(index))
                break
    return exclude

include_indexes = list_index_to_exclude(coco_classes, class_list)
include_indexes

person
car
motorcycle
bus
truck


['0', '2', '3', '5', '7']

Скачивание датасета

In [12]:
import fiftyone as fo
import fiftyone.zoo as foz

dataset = foz.load_zoo_dataset(
    "coco-2017",
    # splits=["test", "validation", "train"],
    # splits=["validation"],
    classes=class_list,
    include_licence=False,
    only_matching=True,
    persistent=True,
)

# Visualize the dataset in the FiftyOne App
# session = fo.launch_app(dataset)

Found annotations at '/home/mosminin/fiftyone/coco-2017/raw/instances_train2017.json'
Sufficient images already downloaded
Existing download of split 'train' is sufficient
Found annotations at '/home/mosminin/fiftyone/coco-2017/raw/instances_val2017.json'
Sufficient images already downloaded
Existing download of split 'validation' is sufficient
Test split is unlabeled; ignoring classes requirement
Found test info at '/home/mosminin/fiftyone/coco-2017/raw/image_info_test2017.json'
Images already downloaded
Existing download of split 'test' is sufficient
Ignoring unsupported parameter 'include_licence' for importer type <class 'fiftyone.utils.coco.COCODetectionDatasetImporter'>
Ignoring unsupported parameter 'persistent' for importer type <class 'fiftyone.utils.coco.COCODetectionDatasetImporter'>
Loading existing dataset 'coco-2017'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


Проверка имспользуемых классов

In [13]:
print(dataset.default_classes)

['0', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', '12', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', '26', 'backpack', 'umbrella', '29', '30', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', '45', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', '66', 'dining table', '68', '69', 'toilet', '71', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', '83', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


Экспорт датасета в формат Yolo

In [15]:
import fiftyone as fo

export_dir = "/home/mosminin/fiftyone/coco-2017/"
label_field = "ground_truth"  # for example

# The splits to export
splits = ["train", "validation", "test"]

# All splits must use the same classes list
classes = class_list

# Export the splits
for split in splits:
    split_view = dataset.match_tags(split)
    split_view.export(
        export_dir=export_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field=label_field,
        split=split,
        classes=classes,
    )

Directory '/home/mosminin/fiftyone/coco-2017/' already exists; export will be merged with existing files
 100% |█████████████| 69682/69682 [1.4m elapsed, 0s remaining, 774.5 samples/s]      
Directory '/home/mosminin/fiftyone/coco-2017/' already exists; export will be merged with existing files
 100% |███████████████| 2951/2951 [5.1s elapsed, 0s remaining, 671.9 samples/s]       
Directory '/home/mosminin/fiftyone/coco-2017/' already exists; export will be merged with existing files
 100% |█████████████| 40670/40670 [59.4s elapsed, 0s remaining, 1.2K samples/s]       


## Добавление изображений без классов для снижения ложного распознования

# Читаем список файлов для обработки

In [16]:
from pathlib import Path
data_type = 'train'
files = Path(f'/home/mosminin/fiftyone/coco-2017/labels/{data_type}/').glob('*.txt')
img_path = f'/home/mosminin/fiftyone/coco-2017/images/{data_type}'
name_list = list(map(str, files))
print(len(name_list), name_list[:3], sep='\n')

69682
['/home/mosminin/fiftyone/coco-2017/labels/train/000000421218.txt', '/home/mosminin/fiftyone/coco-2017/labels/train/000000343542.txt', '/home/mosminin/fiftyone/coco-2017/labels/train/000000384015.txt']


## Обработчик

# Ищем аннотации с неиспользуемыми классами и переименовываем .txt и соответствующий .jpg

In [17]:
import multiprocessing, os
from multiprocessing import Value
from time import time, sleep

files = name_list

p_count = multiprocessing.Value('i', 0)
t_count = multiprocessing.Value('f', time())

done_list = []

def process_file(file_name):
    ###
    # t_count.value = time() - t_count.value
    # p_count.value += 1
    # print(str(p_count.value)+'\n')
    ###
    file = open(file_name, 'r')
    lines = file.readlines()
    rename = True
    data = []
    for each in lines:
        data.append(int(each.split()[0]))
        # if each.split()[0] in [exclude_list]:
        if sum(data) > 0:
            rename = False
    if rename:        
        name = Path(file_name)
        # print(f"{name.stem}{name.suffix}_")
        name.rename(Path(name.parent, f"{name.stem}{name.suffix}_"))
        
        txt_name = Path(file_name)
        rname = Path(img_path, f'{txt_name.stem}.jpg')
        try:
            to_rename = Path(rname.parent, f"{txt_name.stem}{rname.suffix}_")
            os.rename(rname, to_rename)
        except FileNotFoundError:
            print(f'{to_rename} not found\n')
            pass
    file.close()

In [25]:
def process_indexes(name_list=name_list):
    ###
    # t_count.value = time() - t_count.value
    # p_count.value += 1
    # print(str(p_count.value)+'\n')
    ###
    for file_name in name_list:
        file = open(file_name, 'r')
        lines = file.readlines()
        data = []
        for each in lines:
            if each.split()[0] not in data:
                print(each.split()[0]+'\r'+' '*10)
                data.append(each.split()[0])
        file.close()
    return data

In [ ]:
indexes = process_indexes()

In [27]:
indexes

['4', '1']

In [ ]:
# with multiprocessing.Pool(4) as pool:
#     pool.map(process_file, files, 1);